# Fine-Grained Access Control (FGAC) with IBM Data Engine on COS

This notebook demonstrates the setup and usage of fine-grained access inside data lakes on IBM COS buckets. It leverages the Python package `cosacess`, which hides a good amount of complexity and inconsistency of IBM Cloud APIs and SDKs needed to conduct the setup and management of fine-grained access on IBM COS. It also reduced the user input to the minimum information required. You will not have to hassle with instance IDs, CRNs or endpoints. Just bring your COS bucket name and object prefixes. That's it.

This demo also uses IBM Data Engine as a data lake service that can process and query data lakes on COS.

The purpose of the `cosaccess` package this notebook demonstration is to give you a quick start for your own data lake projects on IBM COS.

You can find more details and the methods provided in `cosacess` [here](https://github.com/IBM-Cloud/sql-query-clients/tree/master/cosaccess#cosaccessmanager-method-list).

**Note:** There is a second variand of this same demo that works with plain `cosaccess` package without connecting it and using it through a data lake service like IBM Data Engine. If you are inteterested to see the standalone `cosaccess` demo please refer to "[COS FGAC Demo.ipynb](https://github.com/IBM-Cloud/sql-query-clients/blob/master/cosaccess/COS%20FGAC%20Demo.ipynb)".

<a id="toc"></a>
## Table of contents
1. [Library Set Up](#setup)<br>
2. [Data Engine FGAC Helper](#demo_setup)<br>
3. [Demo Set Up](#data_engine_helper)<br>
    * [Set up the demo data on COS](#demo_data)<br>
    * [SCreate the table definitions for the demo generated data](#demo_tables)<br>
    * [Set up access groups to use for fine-grained access control](#access_groups)<br>
    * [Set up the fine-grained COS access policies for the demo](#access_policies)<br>
    * [Set up Service IDs with individual API Keys for each access group](#service_ids)<br>
4. [COS Access Enforcement Demo](#access_demo)<br>
    * [DBAs Access](#dbas_access)<br>
    * [Portfolio Manager Access](#portfolio_manager_access)<br>
    * [Customer Analysts Access](#customer_analysts_access)<br>
    * [Marketing Fullfillment Access](#marketing_fullfillment_access)<br>
5. [Demo Clean Up](#cleanup)<br>
6. [Further Reference](#reference)<br>
    * [IAM Helpers](#iam_helpers)<br>
        * [Users](#user_helpers)<br>
        * [Service IDs](#service_id_helpers)<br>
        * [Access Groups](#access_group_helpers)<br>
    * [COS Helpers](#cos_helpers)<br>

## <a id="setup"></a> 1. Library Set Up
[Home](#toc)

Install and import `cosaccess` and a few other libraries used in the demo:

In [1]:
!set -o pipefail; pip install --upgrade "cosaccess" | { grep -v "already satisfied" || :; }

In [2]:
import os
from cosaccess import CosAccessManager
import getpass
import ibm_boto3
from ibm_botocore.client import Config, ClientError

For all the below interaction with IBM Cloud we need an API Key associated with your account and a COS bucket where we will set up and consume the demo data. Please provide an API key with administrative privileges in your account so that we can create and configure service IDs and access groups for the demo. Please also provide a COS bucket from a COS instance in your account. **Furthermore** we need a **standard plan** instance of IBM Data Engine in your account. (Lite plan instances are not sufficient for this demo because they don't allow for table cataloging.) Please provide the instance CRN of your Data Engine instance.

In [3]:
try:
    apikey = os.environ["FGACDEMO_APIKEY"]
except KeyError:
    apikey = getpass.getpass("Please enter your IBM Cloud API Key (hit enter): ")
try:
    bucket = os.environ["FGACDEMO_BUCKET"]
except KeyError:
    bucket = input("Please enter an existing COS bucket in your account to use for the demo: ")
try:
    dataengine_id = os.environ["FGACDEMO_DATAENGINE_INSTANCE"]
except KeyError:
    dataengine_id = input("Please the instance CRN of your IBM Data Engine instance: ")
print("We will be using your API Key {} and your bucket {} for this demo. We will be using Data Engine instance {} to work with the data lake data".format("*" * len(apikey), bucket, dataengine_id))

We will be using your API Key ******************************************** and your bucket fgac-lake for this demo. We will be using Data Engine instance crn:v1:bluemix:public:sql-query:us-south:a/d86af7367f70fba4f306d3c19c938f2f:cd98dc40-e7b1-40c0-9d7b-d988072c2699:: to work with the data lake data


Initialize the `cosaccess` client:

In [83]:
cosaccess = CosAccessManager(apikey=apikey)

Initialize the `ibmcloudsql` for Data Engine:

In [84]:
import ibmcloudsql
endpoint = cosaccess.get_cos_endpoint(bucket)
region = endpoint[11:endpoint[11:].find(".")+11]
default_target = "cos://" + region + "/" + bucket + "/temp"
data_engine = ibmcloudsql.SQLQuery(apikey, dataengine_id, client_info='COS FGAC Demo',
                                   target_cos_url=default_target, max_concurrent_jobs=4, max_tries=3 )    
data_engine.logon()

## 2. <a id="data_engine_helper"></a> Data Engine FGAC Helper
To streamline the usage of IBM Data Engine together with the `cosaccess` package we create a little helper class that abstracts the access managment methods found in `cosaccess` on a level of data lake tables defined in a table catalog (in IBM Data Engine). This helper and the demo further down uses the `ibmcloudsql` package, which is the client SQL for IBM Data Engine. So we set that package up first.

In [85]:
!set -o pipefail; pip install --upgrade "ibmcloudsql" | { grep -v "already satisfied" || :; }

In [86]:
import pandas as pd
class DataEngineAccessHelper:
    def __init__(self, cosAccessManager:CosAccessManager, sqlClient:ibmcloudsql.SQLQuery):
        self._cosaccess = cosAccessManager
        self._sqlClient = sqlClient
        
    def get_policies_for_tables(self, table_names:list, roles:list = None):
        policies = []
        for table_name in table_names:
            location = self._sqlClient.get_table_details(table_name)["location"]
            bucket = location.split("/")[3]
            prefix = "/".join(location.split("/")[4:])
            table_policies = self._cosaccess.get_policies_for_cos_bucket(cosBucket=bucket, prefix=prefix, roles=roles)
            policies.append(table_policies)
        return pd.concat(policies).drop_duplicates(subset=["policy_id"])
    
    def grant_table_access(self, table_names:list, roles:list, user_name = None, user_id = None, service_id = None,
                           service_id_name = None, access_group_name = None, access_group_id = None):
        prefixes = []
        for table_name in table_names:
            location = self._sqlClient.get_table_details(table_name)["location"]
            bucket = location.split("/")[3]
            prefixes.append("/".join(location.split("/")[4:]))
        cos_instance = self._cosaccess.get_cos_instance_id(bucket)
        return self._cosaccess.grant_bucket_access(roles=roles, cos_bucket=bucket, prefixes=prefixes, user_name=user_name, user_id=user_id,
                                                   service_id=service_id, service_id_name=service_id_name,
                                                   access_group_name=access_group_name, access_group_id=access_group_id)

In [87]:
data_engine_access = DataEngineAccessHelper(cosaccess, data_engine)

## <a id="demo_setup"></a> 3. Demo Set Up
[Home](#toc)

#### <a id="demo_data"></a> Generate the demo data on COS
First we generate some test data in parquet files. We use Data Engine for this purpose. We generate a little demo data lake that consists of three different tables
 - Table `customers` with list of customers and their contact information
 - Table `products` with list of products and their price
 - Table `sales` with the individual purchases made by customers of certain amounts of a product

In [88]:
# The following link allows you to open the Cloud console for your Data Engine instance "\
# in case you want to observe the generate SQL jobs.")
x=data_engine.sql_ui_link()

https://dataengine.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/d86af7367f70fba4f306d3c19c938f2f:cd98dc40-e7b1-40c0-9d7b-d988072c2699::


In [89]:
datamarturl = "cos://" + region + "/" + bucket + "/data_mart/"

In [90]:
sql='select col1 as product_id, col2 as product_name, col3 as price from \
                (values (1, "Towel", 8.99), \
                        (2, "Soap", 2.79), \
                        (3, "Washrag", 1.99)) \
    INTO {}products/data.parquet JOBPREFIX NONE STORED AS PARQUET'.format(datamarturl)
jobId = data_engine.submit_sql(sql)
data_engine.wait_for_job(jobId)
data_engine.rename_exact_result(jobId)

sql='select col1 as customer_id, col2 as customer_name, col3 as address, col4 as email from \
                (values (1, "Daniel", "1 Duck Street, 00815 Entenhausen", "daniel@entenhausen.de"),\
                        (2, "Torsten", "742 Evergreen Terrace, 80085 Springfield", "torsten@simpsons.de")) \
    INTO {}customers/data.parquet JOBPREFIX NONE STORED AS PARQUET'.format(datamarturl)
jobId = data_engine.submit_sql(sql)
data_engine.wait_for_job(jobId)
data_engine.rename_exact_result(jobId)

sql='select col1 as sales_id, col2 as sales_time, col3 as customer_id, col4 as product_id, col5 as amount from \
                (values (1, timestamp("2023-11-16T09:24:45.652Z"),  1, 1, 4), \
                        (2, timestamp("2023-11-16T09:24:45.652Z"),  1, 2, 1), \
                        (3, timestamp("2023-10-16T14:21:00.000Z"),  2, 3, 3)) \
    INTO {}sales/data.parquet JOBPREFIX NONE STORED AS PARQUET'.format(datamarturl)
jobId = data_engine.submit_sql(sql)
data_engine.wait_for_job(jobId)
data_engine.rename_exact_result(jobId)

Show the just generated parquet data files on COS:

In [91]:
cosaccess.get_cos_objects(bucket, prefix="data_mart")

,name,last_modified,owner,size
0,data_mart/customers/data.parquet,2023-11-24 16:54:41.008000+00:00,"{'DisplayName': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d', 'ID': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d'}",1602
1,data_mart/products/data.parquet,2023-11-24 16:54:32.781000+00:00,"{'DisplayName': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d', 'ID': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d'}",1083
2,data_mart/sales/data.parquet,2023-11-24 16:54:48.543000+00:00,"{'DisplayName': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d', 'ID': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d'}",1579


#### <a id="demo_tables"></a> Create the table definitions for the demo generated data
We use Data Engine to create tables. This is very straight forward since Data Engine will automatically infer all the needed information (such as table columns and types) from the parquet data on COS.

In [92]:
data_engine.create_table("products", cos_url=datamarturl+"products/", format_type="parquet", force_recreate=True)
data_engine.create_table("customers", cos_url=datamarturl+"customers/", format_type="parquet", force_recreate=True)
data_engine.create_table("sales", cos_url=datamarturl+"sales/", format_type="parquet", force_recreate=True)

As an example we take a look at the infered table definiton for on of our tables:

In [93]:
data_engine.get_table_details("sales")

{'name': 'sales',
 'type': 'TABLE',
 'creation_time': '2023-11-24T16:59:46.000Z',
 'data_format': 'parquet',
 'location': 'cos://us-south/fgac-lake/data_mart/sales',
 'partitioning_columns': [],
 'viewStatement': None,
 'columns': [{'name': 'sales_id', 'type': 'integer', 'nullable': True},
  {'name': 'sales_time', 'type': 'timestamp', 'nullable': True},
  {'name': 'customer_id', 'type': 'integer', 'nullable': True},
  {'name': 'product_id', 'type': 'integer', 'nullable': True},
  {'name': 'amount', 'type': 'integer', 'nullable': True}]}

#### <a id="access_groups"></a> Set up access groups to use for fine-grained access control
We create four access groups to use for four different personas:
 - Customer Analysts that analyze the sales transactions to generate a customer ranking.
 - Portfolio Managers that create and manage the list of products
 - Marketing Fullfillment that sends out customer nurture
 - DBAs that are responsible for the data lake content as a whole

After creating these four groups we list them right away for your information:

In [94]:
cosaccess.create_access_group(access_group_name="customer_analysts")
cosaccess.create_access_group(access_group_name="marketing_fulfillment")
cosaccess.create_access_group(access_group_name="portfolio_managers")
cosaccess.create_access_group(access_group_name="DBAs")
cosaccess.get_access_groups()

,id,name,description,created_at,created_by_id,last_modified_at,last_modified_by_id,href
0,AccessGroupId-9fd8d65d-0148-4c44-860d-89c65f454198,customer_analysts,,2023-11-24T17:00:00Z,IBMid-503E85GQTC,2023-11-24T17:00:00Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-9fd8d65d-0148-4c44-860d-89c65f454198
1,AccessGroupId-228cc825-5e6e-4cd9-933d-769a1e930412,DBAs,,2023-11-24T17:00:01Z,IBMid-503E85GQTC,2023-11-24T17:00:01Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-228cc825-5e6e-4cd9-933d-769a1e930412
2,AccessGroupId-e41ad631-6669-4fb4-86b2-d2a49ac7cf74,marketing_fulfillment,,2023-11-24T17:00:00Z,IBMid-503E85GQTC,2023-11-24T17:00:00Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-e41ad631-6669-4fb4-86b2-d2a49ac7cf74
3,AccessGroupId-687115d9-fe10-4e8d-893e-8b1933b4fcb4,portfolio_managers,,2023-11-24T17:00:00Z,IBMid-503E85GQTC,2023-11-24T17:00:00Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-687115d9-fe10-4e8d-893e-8b1933b4fcb4
4,AccessGroupId-PublicAccess,Public Access,"This group includes all users and service IDs by default. All group members, including unauthenticated users, are given public access to any resources that are defined in the policies for the group.",2019-02-23T12:00:00Z,iam-ServiceId-bf540d3e-02b4-4c74-9b00-267936f4c009,,,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-PublicAccess


#### <a id="access_policies"></a> Set up the fine-grained table access policies for the demo

For the demo we apply some governance assumption as follows:
 - Portfolio managers need to be able to read and write the list of products in the `products` table. They don't have a need to know anything about the sales transactions or customer contact details.
 - Customer analysts need to be able to read the sales transactions in the `sales` table and the product details in the `products` table. But they don't have a need to know the customer contact details or product details.
 - Marketing fullfilment needs to read the customer contact details in the `customer` table to send out customer nurture and campaign material to customers. They don't have a need to know any other tables contents.
 - The DBAs need read and write access to all three tables in the data mart.

The following four commands grant four fine-grained table access policies to the access groups that we have created beforehand for these different personas. We then list the new table access policies for your information.

In [104]:
data_engine_access.grant_table_access(table_names=["products"], roles=["Reader", "Writer"], access_group_name="portfolio_managers")
data_engine_access.grant_table_access(table_names=["sales", "products"], roles=["Reader"], access_group_name="customer_analysts")
data_engine_access.grant_table_access(table_names=["customers"], roles=["Reader"], access_group_name="marketing_fulfillment")
data_engine_access.grant_table_access(table_names=["products", "sales", "customers"], roles=["Reader", "Writer"], access_group_name="DBAs")

data_engine_access.get_policies_for_tables(table_names=["products", "sales", "customers"], roles=["Reader", "Writer"])

,instance,bucket,paths,roles,user,service_id,access_group,access_group_id,iam_id,other_subject,policy_id
0,0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d,fgac-lake,"[data_mart/products*, data_mart/sales*, data_mart/customers*]","[Reader, Writer]",,,DBAs,AccessGroupId-228cc825-5e6e-4cd9-933d-769a1e930412,,,123d7600-8f5d-439f-9201-b3f320419df6
1,0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d,fgac-lake,"[data_mart/sales*, data_mart/products*]",[Reader],,,customer_analysts,AccessGroupId-9fd8d65d-0148-4c44-860d-89c65f454198,,,811ae0d7-c5a9-4a49-96c9-8c388ba4deef
2,0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d,fgac-lake,data_mart/products*,"[Reader, Writer]",,,portfolio_managers,AccessGroupId-687115d9-fe10-4e8d-893e-8b1933b4fcb4,,,f0081d24-6d22-416c-8948-4e327452cee0
1,0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d,fgac-lake,data_mart/customers*,[Reader],,,marketing_fulfillment,AccessGroupId-e41ad631-6669-4fb4-86b2-d2a49ac7cf74,,,5fe99ecc-5787-4540-9bae-28a3d63d6359


#### <a id="service_ids"></a> Set up Service IDs with individual API Keys for each access group
We will use these service IDs further down to demonstrate the effects of the fine-grained access policies when the data lake data is accessed. We create each service ID and then assign it to the according access group. We also retrieve and memorize the API Key for each service ID for later. We then list the new service IDs for your information.

In [96]:
sid = cosaccess.create_service_id(service_id_name="portfolio_managers Service ID", with_apikey=True)
cosaccess.add_member_to_access_group(access_group_name="portfolio_managers", service_id_name="portfolio_managers Service ID")
portfolio_managers_apikey=sid["apikey"]["apikey"]

sid = cosaccess.create_service_id(service_id_name="customer_analysts Service ID", with_apikey=True)
cosaccess.add_member_to_access_group(access_group_name="customer_analysts", service_id_name="customer_analysts Service ID")
customer_analysts_apikey=sid["apikey"]["apikey"]

sid = cosaccess.create_service_id(service_id_name="marketing_fulfillment Service ID", with_apikey=True)
cosaccess.add_member_to_access_group(access_group_name="marketing_fulfillment", service_id_name="marketing_fulfillment Service ID")
marketing_fulfillment_apikey=sid["apikey"]["apikey"]

sid = cosaccess.create_service_id(service_id_name="DBAs Service ID", with_apikey=True)
cosaccess.add_member_to_access_group(access_group_name="DBAs", service_id_name="DBAs Service ID")
dbas_apikey=sid["apikey"]["apikey"]

cosaccess.get_service_ids()

,id,iam_id,entity_tag,crn,locked,created_at,created_by,modified_at,account_id,name,unique_instance_crns
0,ServiceId-fda19091-cd64-498c-883c-0ee3a27d0a52,iam-ServiceId-fda19091-cd64-498c-883c-0ee3a27d0a52,1-147f28085d11e1c62ae364c04f05fb82,crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-fda19091-cd64-498c-883c-0ee3a27d0a52,False,2023-11-24T17:01+0000,IBMid-503E85GQTC,2023-11-24T17:01+0000,fcb2ad051f2c0819da9b2c9596977a14,customer_analysts Service ID,[]
1,ServiceId-51e1270c-7ef2-4459-9527-f7ab542a2dee,iam-ServiceId-51e1270c-7ef2-4459-9527-f7ab542a2dee,1-81d28140776651edb0607c970f4fdcd0,crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-51e1270c-7ef2-4459-9527-f7ab542a2dee,False,2023-11-24T17:01+0000,IBMid-503E85GQTC,2023-11-24T17:01+0000,fcb2ad051f2c0819da9b2c9596977a14,DBAs Service ID,[]
2,ServiceId-f7d1917a-5dbd-47a9-80ae-f1f3903c046f,iam-ServiceId-f7d1917a-5dbd-47a9-80ae-f1f3903c046f,1-db4ba194bc0061b8aab63d5a690fdb3d,crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-f7d1917a-5dbd-47a9-80ae-f1f3903c046f,False,2023-11-24T17:01+0000,IBMid-503E85GQTC,2023-11-24T17:01+0000,fcb2ad051f2c0819da9b2c9596977a14,marketing_fulfillment Service ID,[]
3,ServiceId-2a672594-d2e7-4111-8f5f-ed4d814dcfc1,iam-ServiceId-2a672594-d2e7-4111-8f5f-ed4d814dcfc1,1-8f1385b12f24399706c1c0b48373d590,crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-2a672594-d2e7-4111-8f5f-ed4d814dcfc1,False,2023-11-24T17:01+0000,IBMid-503E85GQTC,2023-11-24T17:01+0000,fcb2ad051f2c0819da9b2c9596977a14,portfolio_managers Service ID,[]


As an example we show the access group members for one access group, which should comprise exactly the according service ID that we just created and added to that access group:

In [97]:
cosaccess.get_access_group_members(access_group_name="portfolio_managers")

,iam_id,type,membership_type,href,created_at,created_by_id,service_id_name,user_name
0,iam-ServiceId-2a672594-d2e7-4111-8f5f-ed4d814dcfc1,service,static,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-687115d9-fe10-4e8d-893e-8b1933b4fcb4/members/iam-ServiceId-2a672594-d2e7-4111-8f5f-ed4d814dcfc1,2023-11-24T17:01:22Z,IBMid-503E85GQTC,portfolio_managers Service ID,


## <a id="access_demo"></a> 4. COS Access Enforcement Demo
[Home](#toc)

After we have set up and hardened the data lake data on COS in the previous section, we will now in this section excercise different read and write operations with the different personas on the data in order to demonstrate that our access governance policies are properly enforced with any access.

We are using ibm_boto3 library to perform the different COS access operations.

#### <a id="dbas_access"></a> DBAs Access
Demonstrate that DBAs can read and write objects in all three tables but cannot read or write anything in a different table location:

In [100]:
# We create new boto3 COS clients with the DBAs API key 
# (the API key of the service ID that we created above and added to DBAs access group)
dbas_cosresource = ibm_boto3.resource("s3", ibm_api_key_id=dbas_apikey,
                        ibm_service_instance_id=cosaccess.get_cos_instance_crn(bucket),
                        config=Config(signature_version="oauth"),
                        endpoint_url=cosaccess.get_cos_endpoint(bucket))
dbas_cosclient = ibm_boto3.client("s3", ibm_api_key_id=dbas_apikey, # For delete we need the lower level COS client, unfortunately
                        ibm_service_instance_id=cosaccess.get_cos_instance_crn(bucket),
                        config=Config(signature_version="oauth"),
                        endpoint_url=cosaccess.get_cos_endpoint(bucket))

dbas_cosresource.Object(bucket, "data_mart/products/plumpf.parquet").put(Body="")
print("Wrote data in products table successfully")
dbas_cosresource.Object(bucket, "data_mart/products/plumpf.parquet").get()
print("Read data in products table successfully")
dbas_cosclient.delete_object(Bucket=bucket, Key="data_mart/products/plumpf.parquet")
print("Deleted data from products table successfully")
try:
    dbas_cosresource.Object(bucket, "data_mart/other_table/plumpf.parquet").get()
except ClientError as e:
    print("Works as designed for other_table: " + str(e))
try:
    dbas_cosresource.Object(bucket, "data_mart/other_table/plumpf.parquet").put(Body="")
except ClientError as e:
    print("Works as designed for other_table: " + str(e))

Wrote data in products table successfully
Read data in products table successfully
Deleted data from products table successfully
Works as designed for other_table: An error occurred (AccessForbidden) when calling the GetObject operation: Access Forbidden
Works as designed for other_table: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied


#### <a id="portfolio_manager_access"></a> Portfolio Manager Access
Demonstrate that portfolio managers can read objects in the `products` table and that they cannot read data in the other tables of the data mart:

In [101]:
# We create a new boto3 COS client with the portfolio managers API key 
# (the API key of the service ID that we created above and added to portfolio_managers access group)
portfolio_managers_cosresource = ibm_boto3.resource("s3", ibm_api_key_id=portfolio_managers_apikey,
                        ibm_service_instance_id=cosaccess.get_cos_instance_crn(bucket),
                        config=Config(signature_version="oauth"),
                        endpoint_url=cosaccess.get_cos_endpoint(bucket))

portfolio_managers_cosresource.Object(bucket, "data_mart/products/data.parquet").get()
print("Read data in products table successfully")
try:
    portfolio_managers_cosresource.Object(bucket, "data_mart/sales/data.parquet").get()
except ClientError as e:
    print("Works as designed for sales table: " + str(e))
try:
    portfolio_managers_cosresource.Object(bucket, "data_mart/customers/data.parquet").get()
except ClientError as e:
    print("Works as designed for customers table: " + str(e))

Read data in products table successfully
Works as designed for sales table: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied
Works as designed for customers table: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied


#### <a id="customer_analysts_access"></a> Customer Analysts Access
Demonstrate that customer analysts can only read `products` and `sales` tables but cannot write them, and demonstrate that they cannot read the `customers` table:

In [105]:
# We create a new boto3 COS client with the customer analysts API key 
# (the API key of the service ID that we created above and added to customer_analysts access group)
customer_analysts_cosresource = ibm_boto3.resource("s3", ibm_api_key_id=customer_analysts_apikey,
                        ibm_service_instance_id=cosaccess.get_cos_instance_crn(bucket),
                        config=Config(signature_version="oauth"),
                        endpoint_url=cosaccess.get_cos_endpoint(bucket))

customer_analysts_cosresource.Object(bucket, "data_mart/products/data.parquet").get()
print("Read data in products table successfully")
customer_analysts_cosresource.Object(bucket, "data_mart/sales/data.parquet").get()
print("Read data in sales table successfully")
try:
    customer_analysts_cosresource.Object(bucket, "data_mart/products/data.parquet").put(Body="")
except ClientError as e:
    print("Works as designed for products table: " + str(e))
try:
    customer_analysts_cosresource.Object(bucket, "data_mart/sales/data.parquet").put(Body="")
except ClientError as e:
    print("Works as designed for sales table: " + str(e))
try:
    customer_analysts_cosresource.Object(bucket, "data_mart/customers/data.parquet").get()
except ClientError as e:
    print("Works as designed for customers table: " + str(e))

Read data in products table successfully
Read data in sales table successfully
Works as designed for products table: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
Works as designed for sales table: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
Works as designed for customers table: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied


#### <a id="marketing_fullfillment_access"></a> Marketing Fullfillment Access
Demonstrate that marketing fullfilment can only read (but not write) `customers` table and that it cannot read any other table in the data mart:

In [106]:
# We create a new boto3 COS client with the marketing fullfillment API key 
# (the API key of the service ID that we created above and added to marketing_fulfillment access group)
marketing_fulfillment_cosresource = ibm_boto3.resource("s3", ibm_api_key_id=marketing_fulfillment_apikey,
                        ibm_service_instance_id=cosaccess.get_cos_instance_crn(bucket),
                        config=Config(signature_version="oauth"),
                        endpoint_url=cosaccess.get_cos_endpoint(bucket))

marketing_fulfillment_cosresource.Object(bucket, "data_mart/customers/data.parquet").get()
print("Read data in customers table successfully")
try:
    marketing_fulfillment_cosresource.Object(bucket, "data_mart/customers/data.parquet").put(Body="")
except ClientError as e:
    print("Works as designed for customers table: " + str(e))
try:
    marketing_fulfillment_cosresource.Object(bucket, "data_mart/products/data.parquet").get()
except ClientError as e:
    print("Works as designed for products table: " + str(e))
try:
    marketing_fulfillment_cosresource.Object(bucket, "data_mart/sales/data.parquet").get()
except ClientError as e:
    print("Works as designed for sales table: " + str(e))

Read data in customers table successfully
Works as designed for customers table: An error occurred (AccessDenied) when calling the PutObject operation: Access Denied
Works as designed for products table: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied
Works as designed for sales table: An error occurred (AccessDenied) when calling the GetObject operation: Access Denied


## <a id="cleanup"></a> 5. Demo Clean Up
[Home](#toc)

Run all cells in this section to clean up ALL the assets that were created for the demo, including service IDs, table definitions in Data Engine, dummy data on COS and access policies.

Remove all service IDs and their associated API Keys that we created for demo:

In [107]:
cosaccess.delete_service_id(service_id_name="portfolio_managers Service ID")
cosaccess.delete_service_id(service_id_name="customer_analysts Service ID")
cosaccess.delete_service_id(service_id_name="marketing_fulfillment Service ID")
cosaccess.delete_service_id(service_id_name="DBAs Service ID")

Remove all access policies created for the demo:

In [108]:
for index, row in cosaccess.get_policies_for_cos_bucket("fgac-lake", roles=["Reader", "Writer"]).iterrows():
    for paths in row["paths"]:
        cosaccess.remove_bucket_access(row["policy_id"])
        break

Drop the demo table defintions in Data Engine:

In [112]:
data_engine.drop_table("products")
data_engine.drop_table("customers")
data_engine.drop_table("sales")

'completed'

Remove all generated demo data:

In [109]:
cosaccess.delete_cos_object(cos_bucket=bucket, object_path="data_mart/customers/data.parquet")
cosaccess.delete_cos_object(cos_bucket=bucket, object_path="data_mart/products/data.parquet")
cosaccess.delete_cos_object(cos_bucket=bucket, object_path="data_mart/sales/data.parquet")
cosaccess.get_cos_objects(bucket, prefix="data_mart")

""


Remove all access groups of the demo:

In [110]:
cosaccess.delete_access_group(access_group_name="customer_analysts", force=True)
cosaccess.delete_access_group(access_group_name="marketing_fulfillment", force=True)
cosaccess.delete_access_group(access_group_name="portfolio_managers", force=True)
cosaccess.delete_access_group(access_group_name="DBAs", force=True)

This concludes the core demo of the `cosaccess` package for fine-grained access management and control on COS. The next section shows additional functions that you can fine in the `cosaccess` package beyond the ones we used for the demo above.

## <a id="reference"></a> 6. Further Reference
[Home](#toc)

### <a id="iam_helpers"></a> IAM Helpers
This section shows generic helper methods providing an easy way work with users, service IDs and access groups in your account.

#### <a id="user_helpers"></a> Users
Get a table with all users and their details in your account:

In [111]:
cosaccess.get_users()

,id,iam_id,realm,user_id,firstname,lastname,state,sub_state,email,phonenumber,altphonenumber,photo,account_id,added_on
0,1cc3d91813972d6327bd2cf403595979,IBMid-120000ETQ5,IBMid,torsten@de.ibm.com,Torsten,Steinbach,ACTIVE,,torsten@de.ibm.com,49-7031-16-4686,,https://cloud.ibm.com/avatar/v1/avatar/migrate-bluemix-photos-production/d840b43e-2afb-4b81-a698-7d93dffeecbc.jpg,fcb2ad051f2c0819da9b2c9596977a14,2023-11-20T08:33:21Z
1,9aea43626a5a23c912d7bef71fb5e884,IBMid-503E85GQTC,IBMid,torsten@steinbachnet.de,Torsten,Steinbach,ACTIVE,,torsten@steinbachnet.de,1234567890,,https://cloud.ibm.com/avatar/v1/avatar/migrate-bluemix-photos-production/a529984f-95d9-44a4-887c-5dc23da41af5.jpg,fcb2ad051f2c0819da9b2c9596977a14,2018-01-25T08:18:08.165Z


Retrieve the IAM ID for a user name:

In [12]:
cosaccess.get_user_iam_id("torsten@de.ibm.com")

'IBMid-120000ETQ5'

Retrieve the user name for an IAM ID:

In [13]:
cosaccess.get_user_name(iam_id='IBMid-120000ETQ5')

'Torsten Steinbach torsten@de.ibm.com'

#### <a id="service_id_helpers"></a> Service IDs
Get a table with all service IDs and their details in your account:

In [14]:
cosaccess.get_service_ids()

,id,iam_id,entity_tag,crn,locked,created_at,created_by,modified_at,account_id,name,description,unique_instance_crns
0,ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e,iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e,1-8320c262d3a390bfbdec644c0ef306b3,crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e,False,2023-11-20T08:36+0000,IBMid-503E85GQTC,2023-11-20T08:36+0000,fcb2ad051f2c0819da9b2c9596977a14,Plumpf Service ID,,[]


Retrieve the service ID name for an IAM ID:

In [10]:
cosaccess.get_service_id_name(iam_id='iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e')

'Plumpf Service ID'

Retrieve the IAM ID for a service ID name:

In [20]:
cosaccess.get_service_id_iam_id('Plumpf Service ID')

'iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e'

Retrieve all detail attributes of a service ID:

In [21]:
cosaccess.get_service_id_details(service_id_name='Plumpf Service ID')

{'id': 'ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e',
 'iam_id': 'iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e',
 'entity_tag': '1-8320c262d3a390bfbdec644c0ef306b3',
 'crn': 'crn:v1:bluemix:public:iam-identity::a/fcb2ad051f2c0819da9b2c9596977a14::serviceid:ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e',
 'locked': False,
 'created_at': '2023-11-20T08:36+0000',
 'created_by': 'IBMid-503E85GQTC',
 'modified_at': '2023-11-20T08:36+0000',
 'account_id': 'fcb2ad051f2c0819da9b2c9596977a14',
 'name': 'Plumpf Service ID',
 'description': '',
 'unique_instance_crns': [],
 'Etag': '1-8320c262d3a390bfbdec644c0ef306b3'}

Creating and deleting service IDs:

In [24]:
temp_service_id = cosaccess.create_service_id(service_id_name='Temp Service ID', with_apikey=True)
print("Created service ID \"{}\" with name \"{}\" and API Key \"{}\"."
      .format(temp_service_id["id"], temp_service_id["name"], temp_service_id["apikey"]["apikey"]))
cosaccess.delete_service_id(service_id_name='Temp Service ID')
print("Deleted service ID \"{}\".".format(temp_service_id["id"]))

Created service ID "ServiceId-29909fd3-a924-4235-8eb9-5a9f1fadb343" with name "Temp Service ID" and API Key "61cAPJhJ2hTRfNJTyJHdC3ciUWDiI3j1Gf6Icuhd2lKM".
Deleted service ID "ServiceId-29909fd3-a924-4235-8eb9-5a9f1fadb343".


#### <a id="access_group_helpers"></a> Access Groups
Get a table with all access groups and their details in your account:

In [12]:
cosaccess.get_access_groups()

,id,name,description,created_at,created_by_id,last_modified_at,last_modified_by_id,href
0,AccessGroupId-fe28f512-17fe-45d1-b43c-466ab22d31eb,customer_analysts,,2023-11-20T08:31:49Z,IBMid-503E85GQTC,2023-11-20T08:31:49Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-fe28f512-17fe-45d1-b43c-466ab22d31eb
1,AccessGroupId-19244424-aadf-4287-95d2-e6ce88fc9fc0,DBAs,,2023-11-20T08:32:12Z,IBMid-503E85GQTC,2023-11-20T08:32:12Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-19244424-aadf-4287-95d2-e6ce88fc9fc0
2,AccessGroupId-13cd5002-30d7-4e9b-aa84-44bcc09f22bf,marketing_fulfillment,,2023-11-20T08:32:03Z,IBMid-503E85GQTC,2023-11-20T08:32:03Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-13cd5002-30d7-4e9b-aa84-44bcc09f22bf
3,AccessGroupId-8ae914fd-007e-401d-8692-3bdf3735d23a,portfolio_managers,,2023-11-20T08:31:31Z,IBMid-503E85GQTC,2023-11-20T08:31:31Z,IBMid-503E85GQTC,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-8ae914fd-007e-401d-8692-3bdf3735d23a
4,AccessGroupId-PublicAccess,Public Access,"This group includes all users and service IDs by default. All group members, including unauthenticated users, are given public access to any resources that are defined in the policies for the group.",2019-02-23T12:00:00Z,iam-ServiceId-bf540d3e-02b4-4c74-9b00-267936f4c009,,,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-PublicAccess


Retrieve the access group name for an access group ID:

In [15]:
cosaccess.get_access_group_name(access_group_id="AccessGroupId-8ae914fd-007e-401d-8692-3bdf3735d23a")

'portfolio_managers'

Retrieve the ID for an access group name:

In [16]:
cosaccess.get_access_group_id(access_group="portfolio_managers")

'AccessGroupId-8ae914fd-007e-401d-8692-3bdf3735d23a'

Show access group members:

In [27]:
cosaccess.get_access_group_members(access_group_name="portfolio_managers")

,iam_id,type,membership_type,href,created_at,created_by_id,service_id_name,user_name
0,iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e,service,static,https://iam.cloud.ibm.com/v2/groups/AccessGroupId-8ae914fd-007e-401d-8692-3bdf3735d23a/members/iam-ServiceId-b8e19a3c-d1fc-493a-ac1f-e7fbad148e0e,2023-11-20T08:37:16Z,IBMid-503E85GQTC,Plumpf Service ID,


Creating and deleting access groups:

In [6]:
temp_access_group_name = "Temp Access Group"
temp_access_group_id = cosaccess.create_access_group(access_group_name=temp_access_group_name)
print("Created Access Group' \"{}\" with id \"{}\"."
      .format(temp_access_group_name, temp_access_group_id))
cosaccess.delete_access_group(access_group_name=temp_access_group_name, force=True)
print("Deleted service ID \"{}\".".format(temp_access_group_id))

Created Access Group' "Temp Access Group" with id "AccessGroupId-cbfc346b-ff85-46f6-bea2-a148b1630448".
Deleted service ID "AccessGroupId-cbfc346b-ff85-46f6-bea2-a148b1630448".


### <a id="cos_helpers"></a> General COS Helpers

Create an empty object (similar to `touch` shell command), show it in an object listing and then delete it again:

In [14]:
cosaccess.touch_cos_object(cos_bucket=bucket, object_path="test_data/test_object")

In [15]:
cosaccess.get_cos_objects(cos_bucket=bucket, prefix="test_data")

,name,last_modified,owner,size
0,test_data/test_object,2023-11-23 10:14:02.512000+00:00,"{'DisplayName': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d', 'ID': '0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d'}",0


In [16]:
cosaccess.delete_cos_object(cos_bucket=bucket, object_path="test_data/test_object")

Get the instance ID and CRN for a COS bucket:

In [10]:
cosaccess.get_cos_instance_id(bucket)

'0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d'

In [11]:
cosaccess.get_cos_instance_crn(bucket)

'crn:v1:bluemix:public:cloud-object-storage:global:a/fcb2ad051f2c0819da9b2c9596977a14:0ca7988d-6cb0-4ccf-85e3-9d7af75d5e0d::'

Get the endpoint for a COS bucket:

In [9]:
cosaccess.get_cos_endpoint(bucket)

'https://s3.us-south.cloud-object-storage.appdomain.cloud'

List all supported COS endpoints in IBM Cloud:

In [8]:
cosaccess.list_cos_endpoints()

{'cross-region': {'us': {'public': {'us-geo': 's3.us.cloud-object-storage.appdomain.cloud',
    'Dallas': 's3.dal.us.cloud-object-storage.appdomain.cloud',
    'Washington': 's3.wdc.us.cloud-object-storage.appdomain.cloud',
    'San Jose': 's3.sjc.us.cloud-object-storage.appdomain.cloud'},
   'private': {'us-geo': 's3.private.us.cloud-object-storage.appdomain.cloud',
    'Dallas': 's3.private.dal.us.cloud-object-storage.appdomain.cloud',
    'Washington': 's3.private.wdc.us.cloud-object-storage.appdomain.cloud',
    'San Jose': 's3.private.sjc.us.cloud-object-storage.appdomain.cloud'},
   'direct': {'us-geo': 's3.direct.us.cloud-object-storage.appdomain.cloud',
    'Dallas': 's3.direct.dal.us.cloud-object-storage.appdomain.cloud',
    'Washington': 's3.direct.wdc.us.cloud-object-storage.appdomain.cloud',
    'San Jose': 's3.direct.sjc.us.cloud-object-storage.appdomain.cloud'}},
  'eu': {'public': {'eu-geo': 's3.eu.cloud-object-storage.appdomain.cloud',
    'Amsterdam': 's3.ams.eu.cloud

### Author
**Torsten Steinbach**, Torsten is IBM's CTO for Big Data in Cloud.